## Machine Translation
- Rule-based MT (RBMT)
- Statistical MT (SMT)
  - Phrased based MT (PBMT)
- 통계기반 MT부터 corpus가 있다면 모델 생성 가능
- NMT

## Seq2Seq
- Encoder - Decoder - Generator
- Dataset
  - D = {x_i, y_i}_i=1~N
  - x_i = {x_1, ... , x_m}, y_i = {y_0, ... , y_n}
  - y_0 = \<BOS\>, y_n = \<EOS\>
- Applications
  - NMT
  - Chatbot
  - Summarization
  - Automatic Speech Recognition
  - Lip Reading
  - Image Captioning

### Encoder
- 문장을 하나의 벡터로 만들어주는 역할
- |x_i| = (bs, m, |v|), |y_i| = (bs, n, |v|)
- h_t = RNN_enc( emb_enc(x_t), h_t-1 ), |x_t| = (bs, 1, |v|), |h_t| = (bs, 1, hs)
- h_1:m = [h_1; ... ; h_m], |h_1:m| = (bs, m, hs)
- bi-directional인 경우는 2 * hs
- Encoder는 train/test시 항상 문장 전체를 받음
  - Encoder만 봤을 때는 non-auto-regressive task임
  - bi-directional RNN 사용 가능

### Decoder
- Encoder의 마지막 hs가 Decoder의 initial hs가 됌
- Auto-regressive task이므로 uni-directional RNN 사용해야함
- Encoder의 hs와 Decoder의 hs 차이를 조절해야함

### Generator
- y_hat_t = softmax(h_t * W_gen)
- |h_t| = (bs, 1, hs), |y_hat_t| = (bs, 1, |v|), |W_gen| = (hs, |v|)

### Attention
- Encoder, Decoder는 RNN 구조로 구성되기에, timestep이 커질수록 정보 소실이 발생
- 미분 가능한 Key-value Function, ex. d = {k:v, k:v , ...} -> d[q]= v
- dict에서는 적합한 key가 주어지지 않을 경우 error 발생
- 미분가능한 key-value Function은 주어진 Query와 key의 유사도에 따라 Value 변환
- Encoder에 query를 날려서 필요한 정보를 보완
- Query를 잘 만들어내는 과정을 학습
- Attention in Seq2Seq
  - Query: 현재 time-step의 decoder output, 현재 step의 decoder 상태
  - Keys: 각 time-step 별 encoder output
  - Values: 각 time-step 별 encoder output
  - Query를 날리는 과정에서 Linear Transform(Linear Layer)을 통해 변환
  - Linear Transform이 학습
  - ex. '강남역에서 가장 회식 하기 좋은 오리고기 맛있는 집은 어디야?'
  - -> '강남역 오리고기 회식장소 맛집' 머리속의 생각을 검색하기 좋게 Query로 변환
- Batch Matrix Multiplication (BMM)
  - z = torch.bmm(x, y)
  - (batch_size, n, k) * (batch_size, k, m) = (batch_size, n, m)
- Equation
  - w = softmax(h_t . W_a . Keys), h_t . W_a = Q
  - c = w . Value

#### Masking on Attention
- 문장을 mini-batch로 학습하기 때문에, 문장간 sequence 차이 발생
- \<PAD\>를 채워서 학습하는 것은 성능 저하 가능성 존재
- \<PAD\> 뭉치에 의해 attention 계산 과정에서 \<PAD\>의 weight가 계산 되는 문제
- \<PAD\>에는 attention weight가 가지 않도록 해줘야함
- \<PAD\> 모양에 mask를 처리함, Q . K 결과에 mask 모양으로 -inf 처리
- 이를 통해 attention weight에 \<PAD\> 위치에 0으로

### Input Feeding
- 이전 step의 softmax 처리 전 출력(h_t_tilda)을 word embedding에 concat
- 이전 step softmax 결과가 6:4나 9:1나 같은 결과를 다음 step으로(class 2개 가정)
- 헷갈리는 정보의 손실이 나타남
- 이를 통해 Teacher Forcing의 문제도 일부 해결
